In [7]:
import torch
import json
from transformers import AutoModelForCausalLM, AutoTokenizer
from utils import *
delete_cache()

All __pycache__ directories have been deleted.


In [8]:
model_name = "./output/Qwen3-1.7B/checkpoint-64"

tokenizer = AutoTokenizer.from_pretrained(
    model_name, 
    use_fast=False,
    trust_remote_code=True
)
model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    device_map="auto", 
    dtype=torch.bfloat16
)


In [9]:
prompt = "你是一个香港保险经纪人，你在考试，你需要正确回答考试题目。回答格式举例。答案：E\n解释：...（简单解释一下）"

In [10]:
pred_list = []

with open("./val.jsonl", "r", encoding="utf-8") as f:
    for idx, line in enumerate(f):
        line = line.strip()
        if not line:
            continue

        data = json.loads(line)
        question = data["question"]

        messages = [
            {"role": "system", "content": prompt},
            {"role": "user", "content": question}
        ]

        response = predict(messages, model, tokenizer)

        # 取 response 最后一个字符（防止空字符串）
        last_char = response[-1] if response else None
        pred_list.append(last_char)
        print(f"{idx+1}: {last_char}")

1: A
2: A
3: A
4: B
5: C
6: C
7: D
8: D
9: D
10: C
11: C
12: D
13: B
14: C
15: C
16: C
17: C
18: B
19: C
20: D
21: A
22: D
23: C
24: C
25: D


In [11]:
correct = 0
total = 0

with open("./val.jsonl", "r", encoding="utf-8") as f:
    for idx, line in enumerate(f):
        line = line.strip()
        if not line:
            continue

        data = json.loads(line)
        gold = data["answer"]  # 你已确认是 A / B / C / D

        pred = pred_list[idx]

        total += 1
        if pred == gold:
            correct += 1

accuracy = correct / total if total > 0 else 0.0

print(f"Accuracy: {accuracy:.2f} ({correct}/{total})")

Accuracy: 0.56 (14/25)


In [12]:
# test_texts = {
#     'instruction': prompt,
#     'input': "就人壽保險而言，判斷一項可保性的基本因素可能是a)準投保生命的性別\nb)準投保生命的年齡\nc)準投保生命的健康狀況\nd)較大的投保額"
# }

# instruction = test_texts['instruction']
# input_value = test_texts['input']

# messages = [
#     {"role": "system", "content": f"{instruction}"},
#     {"role": "user", "content": f"{input_value}"}
# ]

# response = predict(messages, model, tokenizer)
# print(response)